In [1]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
from sklearn import preprocessing

bodyfat = pd.read_csv("../data/bodyfat.csv")
X = bodyfat.drop(columns=["BodyFat","Density"])
y = bodyfat["BodyFat"]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state = 10)

kf = KFold(n_splits = 5, shuffle = True, random_state = 10)
cv_fold = np.zeros(len(y_train)).astype(int)

for i, (_, fold_indexes) in enumerate(kf.split(X_train)):
    cv_fold[fold_indexes] = int(i)

In [10]:
print('X_train:')
display(X_train.head())
display(X_train.shape)

print('\ncv_fold:')
display(cv_fold)

print('\ny_train:')
display(y_train.head())
display(y_train.shape)

X_train:


,Age,Weight,Height,Neck,Chest,Abdomen,Hip,Thigh,Knee,Ankle,Biceps,Forearm,Wrist
70,62,167.50,71.50,35.5,97.6,91.5,98.5,56.6,38.6,22.4,31.5,27.3,18.6
191,42,244.25,76.00,41.8,115.2,113.7,112.4,68.5,45.0,25.5,37.1,31.2,19.9
46,40,133.50,67.50,33.6,88.2,73.7,88.5,53.3,34.5,22.5,27.9,26.2,17.3
213,50,194.75,70.75,39.0,103.7,97.6,104.2,60.0,40.9,25.5,32.7,30.0,19.0
169,35,172.75,69.50,37.6,99.1,90.8,98.1,60.1,39.1,23.4,32.5,29.8,17.4


(201, 13)


cv_fold:


array([4, 0, 0, 2, 3, 0, 3, 1, 4, 4, 0, 2, 2, 4, 2, 4, 4, 2, 3, 0, 0, 1,
       3, 2, 1, 2, 1, 3, 3, 2, 4, 4, 2, 4, 3, 0, 3, 2, 1, 0, 4, 1, 2, 1,
       3, 2, 0, 0, 2, 1, 3, 2, 0, 2, 4, 0, 1, 4, 1, 0, 1, 0, 4, 0, 4, 4,
       1, 1, 1, 0, 1, 3, 1, 4, 3, 0, 0, 4, 0, 2, 1, 2, 2, 1, 2, 3, 4, 0,
       4, 4, 1, 0, 4, 3, 3, 1, 3, 2, 0, 1, 4, 1, 0, 1, 3, 2, 3, 4, 3, 0,
       1, 0, 2, 4, 2, 4, 0, 1, 3, 2, 1, 1, 4, 4, 1, 4, 4, 0, 2, 3, 2, 0,
       2, 3, 3, 3, 0, 3, 2, 4, 4, 4, 0, 3, 1, 3, 3, 3, 2, 3, 3, 3, 3, 4,
       0, 0, 4, 4, 4, 1, 0, 2, 1, 0, 4, 2, 3, 1, 1, 2, 1, 0, 1, 0, 3, 0,
       1, 3, 2, 0, 2, 3, 4, 3, 4, 2, 1, 2, 2, 2, 2, 0, 2, 1, 0, 0, 4, 1,
       3, 3, 1])


y_train:


70     24.3
191    38.1
46     10.8
213    18.7
169    16.5
Name: BodyFat, dtype: float64

(201,)

In [37]:
def normalize_data(X_train:np.ndarray, X_test:np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """
    Normalize the training and testing data using StandardScaler.
    
    Parameters:
    - X_train (np.ndarray): Training feature data.
    - X_test (np.ndarray): Testing feature data.
    
    Returns:
    - tuple[np.ndarray, np.ndarray]: Normalized training and testing feature data.
    """
    scaler = preprocessing.StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled

In [ ]:
X_train_norm, X_test_norm = normalize_data(X_train, X_test)
print('X_train_norm:')
display(pd.DataFrame(X_train_norm,columns=X_train.columns).head()   )
display(X_train_norm.shape)
print('Standard deviation:', np.std(X_train_norm, axis=0))


X_train_norm:


,Age,Weight,Height,Neck,Chest,Abdomen,Hip,Thigh,Knee,Ankle,Biceps,Forearm,Wrist
0,1.400681,-0.369547,0.386284,-1.005252,-0.373185,-0.092722,-0.203796,-0.542717,0.012316,-0.378185,-0.269119,-0.682443,0.397447
1,-0.208561,2.184283,1.553849,1.558080,1.743507,1.977039,1.678470,1.656375,2.609490,1.333121,1.562323,1.241899,1.774807
2,-0.369485,-1.500886,-0.651552,-1.778320,-1.503691,-1.752260,-1.557945,-1.152549,-1.651499,-0.322981,-1.446475,-1.225207,-0.979913
3,0.435136,0.537187,0.191690,0.418821,0.360441,0.475996,0.568069,0.085595,0.945675,1.333121,0.123333,0.649794,0.821250
4,-0.771796,-0.194855,-0.132634,-0.150808,-0.192786,-0.157985,-0.257962,0.104075,0.215220,0.173850,0.057924,0.551110,-0.873962


(201, 13)

Standard deviation: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [ ]:
import itertools

def best_subset_selection(X_train:np.ndarray, Y_train:np.ndarray)-> dict:
    """
    Perform best subset selection for linear regression. This function evaluates all possible combinations of features
    and selects the best model for each subset size based on R-squared.

    Parameters:
    - X_train (np.ndarray): Training feature data. Shape (n_samples, n_features).
    - Y_train (np.ndarray): Training target data. Shape (n_samples,).

    Returns:
    - dict: A dictionary where each key is the subset size (as a string) and the value is another dictionary containing:
        - 'best_model': The statsmodels OLS regression results object for the best model.
        - 'best_r2': The R-squared value of the best model.
        - 'best_features': A tuple of feature indices used in the best model.
    
    
    """
    
    # Useful dimensions
    n, p = X_train.shape

    # Initialize dictionary to store the best model for each subset size
    best_model_k = {}
    
    # Create model for no feature
    best_model_k['0'] = {}
    best_model_k['0']['best_model'] = sm.OLS(Y_train, np.ones((len(Y_train),1))).fit()
    best_model_k['0']['best_r2'] = best_model_k['0']['best_model'].rsquared
    best_model_k['0']['best_features'] = []

    for k in range(1, p+1):
        best_r2 = 0
        best_features = [] 
        best_model = []
        
        # Create every model with k features (excluding intercept)
        for feature_combination in itertools.combinations(range(p), k):
            X_train_aux  = sm.add_constant(X_train[:,feature_combination])
            model = sm.OLS(Y_train, X_train_aux).fit()
            if model.rsquared > best_r2:
                best_r2 = model.rsquared
                best_model = model
                best_features = feature_combination

        # Store the best model for this subset size
        best_model_k[str(k)] = {
            'best_model': best_model,
            'best_r2': best_r2,
            'best_features': best_features
        }

    return best_model_k
            
#best_models = best_subset_selection(X_train.values, y_train.values)
        




In [86]:
def foward_stepwise_selection(X_train:np.ndarray, Y_train:np.ndarray)-> dict:
    """
    Function to perform forward stepwise selection for linear regression. This function iteratively adds features
    to the model and selects the best model for each subset size based on R-squared. Only one feature is added at each step.


    Parameters:
    - X_train (np.ndarray): Training feature data. Shape (n_samples, n_features).
    - Y_train (np.ndarray): Training target data. Shape (n_samples,).

    Returns:
    - dict: A dictionary where each key is the subset size (as a string) and the value is another dictionary containing:
        - 'best_model': The statsmodels OLS regression results object for the best model.
        - 'best_r2': The R-squared value of the best model.
        - 'best_features': A tuple of feature indices used in the best model.
    
    
    """
    
    # Useful dimensions
    n, p = X_train.shape

    # Initialize dictionary to store the best model for each subset size
    best_model_k = {}
    
    # Create model for no feature
    best_model_k['0'] = {}
    best_model_k['0']['best_model'] = sm.OLS(Y_train, np.ones((len(Y_train),1))).fit()
    best_model_k['0']['best_r2'] = best_model_k['0']['best_model'].rsquared
    best_model_k['0']['best_features'] = []

    remaining_features = list(range(p))
    current_features = []

    for k in range(1, p+1):
        best_r2 = 0
        best_feature = [] 
        best_model = []
        # Create every model with k features (excluding intercept)
        for feature_combination in itertools.combinations(remaining_features, 1):
            #import pdb; pdb.set_trace()
            feature_combination_list = list(feature_combination)  # Convert tuple to list
            X_train_aux  = sm.add_constant(X_train[:,feature_combination_list + current_features])
            model = sm.OLS(Y_train, X_train_aux).fit()
            if model.rsquared > best_r2:
                best_r2 = model.rsquared
                best_model = model
                best_feature = feature_combination_list[0]

        # Move best feature for k from the remainig_features list to current_features
        remaining_features.remove(best_feature)
        current_features.append(best_feature)

        # Store the best model for this subset size
        best_model_k[str(k)] = {
            'best_model': best_model,
            'best_r2': best_r2,
            'best_features': current_features.copy()
        }

    return best_model_k

best_models_pl = foward_stepwise_selection(X_train.values, y_train.values)

In [ ]:
def lasso_regression_cv(X: np.ndarray, y: np.ndarray, cv_fold: np.ndarray, alphas: list[float]) -> dict:

{'0': {'best_model': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x1f877bd7fb0>,
  'best_r2': np.float64(0.0),
  'best_features': []},
 '1': {'best_model': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x1f877bf89b0>,
  'best_r2': np.float64(0.6404157973526722),
  'best_features': [5]},
 '2': {'best_model': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x1f877bf8c50>,
  'best_r2': np.float64(0.693716876511187),
  'best_features': [5, 1]},
 '3': {'best_model': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x1f877bf8e30>,
  'best_r2': np.float64(0.7100124087139326),
  'best_features': [5, 1, 12]},
 '4': {'best_model': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x1f877bf8f50>,
  'best_r2': np.float64(0.7192801308495235),
  'best_features': [5, 1, 12, 10]},
 '5': {'best_model': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x1f877bf9a30>,
  'best_r2': np.float64(0.72315545

## b